### Criando a conexão

In [2]:
import psycopg2 #biblioteca para conectar ao banco
import pandas as pd #biblioteca para criação e gerenciamento de data frame. Também utilizada para exportar o dataframe para csv
import pandas.io.sql as sqlio #utilizado para integrar o sql com pandas, permitindo exportar os dados direto do banco para um dataframe

In [3]:
conn = psycopg2.connect(    
    database="conectcar",
    user="raul_correa", 
    password="senha@091!",
    host="sdlf-cntcar-redshift-dev.capkkn406tig.us-east-2.redshift.amazonaws.com",
    port="5439"
)

### Bases Habilitadas - fotos por período

#### *09 meses atrás*

In [7]:
#version 3.0
cte_BasePF_Clientes1Placa_09 = """
DROP TABLE IF EXISTS #MesRef_09;
CREATE TABLE #MesRef_09 (MesRefVar DATE);
INSERT INTO #MesRef_09
select TO_DATE(ADD_MONTHS(current_date, -9), 'YYYY-MM-01') as MesRefVar
;

DROP TABLE IF EXISTS #baseHabilitada_09;
CREATE TABLE #baseHabilitada_09 (documento VARCHAR, anomes DATE, agregado VARCHAR, anomesCancelamento DATE);
INSERT INTO #baseHabilitada_09
select distinct
d.documento*1 :: float as documento,
TO_DATE(datahabilitada, 'YYYY-MM-01') as anomes,
a.agregado as agregado,
TO_DATE(b.datacancelamento, 'YYYY-MM-01') as anomesCancelamento
from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as a
left join cntcar_work.tb_conectcar_dbo_adesao_stage as b on a.adesaoid = b.adesaoid
left join cntcar_work.tb_conectcar_dbo_veiculo_stage as c on b.veiculoID = c.veiculo_id
left join conectcar.cntcar_work.tb_conectcar_dbo_cliente_stage as d on b.Clienteid = d.Clienteid
where anomes = (SELECT MesRefVar FROM #MesRef_09)
and len(documento) <= 11
;

DROP TABLE IF EXISTS #flagPlacasas_09;
CREATE TABLE #flagPlacasas_09 (documento VARCHAR, flagPlacas FLOAT);
INSERT INTO #flagPlacasas_09
select
documento,
count (documento) as flagPlacas
from #baseHabilitada_09
group by documento
having flagPlacas > 1
order by flagPlacas desc
;

DROP TABLE IF EXISTS #baseHabilitada_09PFCom1PlacaSemPorto;
CREATE TABLE #baseHabilitada_09PFCom1PlacaSemPorto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_09PFCom1PlacaSemPorto
select
a.documento ::float,
a.anomes,
a.agregado,
a.anomescancelamento
from #baseHabilitada_09 a
left join #flagPlacasas_09 b
on a.documento = b.documento
where b.flagPlacas is NULL
;

DROP TABLE IF EXISTS #baseHabilitada_09Porto;
CREATE TABLE #baseHabilitada_09Porto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_09Porto
select
regexp_replace(c.codigointernoparceiro,'[^0-9]+', '') as Documento,
TO_DATE(datahabilitada, 'YYYY-MM-01') as anomes,
p.agregado,
TO_DATE(a.datacancelamento, 'YYYY-MM-01') as anomescancelamento
from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as bh
left outer join cntcar_work.tb_parceiro_dbo_ativacaocontrato_stage ac on ac.adesaoid = bh.adesaoid
left outer join cntcar_work.tb_parceiro_dbo_contrato_stage c on c.contratoid = ac.contratoid
left outer join cntcar_work.vw_redshift_ciclodevida_ativacao a on a.adesaoid = ac.adesaoid
left outer join cntcar_work.vw_ciclodevida_portifolio p on p.adesaoid = bh.adesaoid
where
upper(p.agregado) = 'PORTO SEGURO'
and upper(p.detalhado) not in ('CARROFACIL','CARRO FACIL','CARRO FÁCIL')
and upper(a.nomeplano) <> 'LOCADORA'
and TO_DATE(datahabilitada, 'YYYY-MM-01') = (SELECT MesRefVar FROM #MesRef_09)
;
DROP TABLE IF EXISTS #flagPlacasasPorto_09;
CREATE TABLE #flagPlacasasPorto_09 (documento VARCHAR, flagPlacas FLOAT);
INSERT INTO #flagPlacasasPorto_09
select
documento,
count(documento) as flagPlacas
from #baseHabilitada_09Porto
group by documento
having flagPlacas > 1
;
DROP TABLE IF EXISTS #baseHabilitada_09PFCom1Porto;
CREATE TABLE #baseHabilitada_09PFCom1Porto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_09PFCom1Porto
select
a.documento::float,
a.anomes,
a.agregado,
a.anomescancelamento
from #baseHabilitada_09Porto a
left join #flagPlacasasPorto_09 b on a.documento = b.documento
where b.flagPlacas is null and len(a.documento) <=11
;
DROP TABLE IF EXISTS #basefinal_09;
CREATE TABLE #basefinal_09 (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #basefinal_09
select * from #baseHabilitada_09PFCom1PlacaSemPorto
union
select * from #baseHabilitada_09PFCom1Porto
;
select distinct * from #basefinal_09
"""

In [8]:
BasePF_Clientes1Placa_09 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_09, conn) 

In [46]:
print()BasePF_Clientes1Placa_09

           documento      anomes      agregado anomescancelamento
0        22429766809  2022-06-01        Básico               None
1        21761856847  2022-06-01      Completo               None
2        17456278806  2022-06-01        Básico               None
3         2332682783  2022-06-01        Básico         2022-10-01
4        41035595869  2022-06-01        Básico               None
...              ...         ...           ...                ...
1232345   1088264816  2022-06-01  Porto Seguro               None
1232346   4374191429  2022-06-01  Porto Seguro               None
1232347  19210383818  2022-06-01  Porto Seguro               None
1232348  40195833821  2022-06-01  Porto Seguro         2022-08-01
1232349  10550945717  2022-06-01  Porto Seguro               None

[1232350 rows x 4 columns]


In [ ]:
excluindo_09 = """
DROP TABLE IF EXISTS #MesRef_09;
DROP TABLE IF EXISTS #baseHabilitada_09;
DROP TABLE IF EXISTS #flagPlacasas_09;
DROP TABLE IF EXISTS #baseHabilitada_09PFCom1Porto;
DROP TABLE IF EXISTS #baseHabilitada_09Porto;
DROP TABLE IF EXISTS #baseHabilitada_09PFCom1PlacaSemPorto;
select top 1 * from cntcar_work.tb_redshift_ciclodevida_cliente_stage
"""
sqlio.read_sql_query(excluindo_09, conn) 

#### *08 meses atrás*

In [40]:
cte_BasePF_Clientes1Placa_08 = """
DROP TABLE IF EXISTS #MesRef_08;
CREATE TABLE #MesRef_08 (MesRefVar DATE);
INSERT INTO #MesRef_08
select TO_DATE(ADD_MONTHS(current_date, -8), 'YYYY-MM-01') as MesRefVar
;

DROP TABLE IF EXISTS #baseHabilitada_08;
CREATE TABLE #baseHabilitada_08 (documento VARCHAR, anomes DATE, agregado VARCHAR, anomesCancelamento DATE);
INSERT INTO #baseHabilitada_08
select distinct
d.documento*1 :: float as documento,
TO_DATE(datahabilitada, 'YYYY-MM-01') as anomes,
a.agregado as agregado,
TO_DATE(b.datacancelamento, 'YYYY-MM-01') as anomesCancelamento
from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as a
left join cntcar_work.tb_conectcar_dbo_adesao_stage as b on a.adesaoid = b.adesaoid
left join cntcar_work.tb_conectcar_dbo_veiculo_stage as c on b.veiculoID = c.veiculo_id
left join conectcar.cntcar_work.tb_conectcar_dbo_cliente_stage as d on b.Clienteid = d.Clienteid
where anomes = (SELECT MesRefVar FROM #MesRef_08)
and len(documento) <= 11
;

DROP TABLE IF EXISTS #flagPlacasas_08;
CREATE TABLE #flagPlacasas_08 (documento VARCHAR, flagPlacas FLOAT);
INSERT INTO #flagPlacasas_08
select
documento,
count (documento) as flagPlacas
from #baseHabilitada_08
group by documento
having flagPlacas > 1
order by flagPlacas desc
;

DROP TABLE IF EXISTS #baseHabilitada_08PFCom1PlacaSemPorto;
CREATE TABLE #baseHabilitada_08PFCom1PlacaSemPorto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_08PFCom1PlacaSemPorto
select
a.documento ::float,
a.anomes,
a.agregado,
a.anomescancelamento
from #baseHabilitada_08 a
left join #flagPlacasas_08 b
on a.documento = b.documento
where b.flagPlacas is NULL
;

DROP TABLE IF EXISTS #baseHabilitada_08Porto;
CREATE TABLE #baseHabilitada_08Porto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_08Porto
select
regexp_replace(c.codigointernoparceiro,'[^0-9]+', '') as Documento,
TO_DATE(datahabilitada, 'YYYY-MM-01') as anomes,
p.agregado,
TO_DATE(a.datacancelamento, 'YYYY-MM-01') as anomescancelamento
from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as bh
left outer join cntcar_work.tb_parceiro_dbo_ativacaocontrato_stage ac on ac.adesaoid = bh.adesaoid
left outer join cntcar_work.tb_parceiro_dbo_contrato_stage c on c.contratoid = ac.contratoid
left outer join cntcar_work.vw_redshift_ciclodevida_ativacao a on a.adesaoid = ac.adesaoid
left outer join cntcar_work.vw_ciclodevida_portifolio p on p.adesaoid = bh.adesaoid
where
upper(p.agregado) = 'PORTO SEGURO'
and upper(p.detalhado) not in ('CARROFACIL','CARRO FACIL','CARRO FÁCIL')
and upper(a.nomeplano) <> 'LOCADORA'
and TO_DATE(datahabilitada, 'YYYY-MM-01') = (SELECT MesRefVar FROM #MesRef_08)
;
DROP TABLE IF EXISTS #flagPlacasasPorto_08;
CREATE TABLE #flagPlacasasPorto_08 (documento VARCHAR, flagPlacas FLOAT);
INSERT INTO #flagPlacasasPorto_08
select
documento,
count(documento) as flagPlacas
from #baseHabilitada_08Porto
group by documento
having flagPlacas > 1
;
DROP TABLE IF EXISTS #baseHabilitada_08PFCom1Porto;
CREATE TABLE #baseHabilitada_08PFCom1Porto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_08PFCom1Porto
select
a.documento::float,
a.anomes,
a.agregado,
a.anomescancelamento
from #baseHabilitada_08Porto a
left join #flagPlacasasPorto_08 b on a.documento = b.documento
where b.flagPlacas is null and len(a.documento) <=11
;
DROP TABLE IF EXISTS #basefinal_08;
CREATE TABLE #basefinal_08 (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #basefinal_08
select * from #baseHabilitada_08PFCom1PlacaSemPorto
union
select * from #baseHabilitada_08PFCom1Porto
;
select distinct * from #basefinal_08
"""

In [22]:
BasePF_Clientes1Placa_08 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_08, conn) 

In [41]:
print(BasePF_Clientes1Placa_08)

           documento      anomes      agregado anomescancelamento
0        34974971859  2022-07-01      Completo               None
1        34146209870  2022-07-01          Itaú               None
2        54257751134  2022-07-01        Básico               None
3         8211878640  2022-07-01          Itaú         2022-11-01
4         5694305779  2022-07-01      COMPLETO         2022-11-01
...              ...         ...           ...                ...
1281277  65678028553  2022-07-01  Porto Seguro               None
1281278   4113600864  2022-07-01  Porto Seguro               None
1281279  25221991888  2022-07-01  Porto Seguro               None
1281280  37737415882  2022-07-01  Porto Seguro         2022-10-01
1281281  67378269549  2022-07-01  Porto Seguro         2023-02-01

[1281282 rows x 4 columns]


In [ ]:
excluindo_08 = """
DROP TABLE IF EXISTS #MesRef_08;
DROP TABLE IF EXISTS #baseHabilitada_08;
DROP TABLE IF EXISTS #flagPlacasas_08;
DROP TABLE IF EXISTS #baseHabilitada_08PFCom1Porto;
DROP TABLE IF EXISTS #baseHabilitada_08Porto;
DROP TABLE IF EXISTS #baseHabilitada_08PFCom1PlacaSemPorto;
select top 1 * from cntcar_work.tb_redshift_ciclodevida_cliente_stage
"""
sqlio.read_sql_query(excluindo_08, conn) 

#### *07 meses atrás*

In [ ]:
cte_BasePF_Clientes1Placa_07 = """
DROP TABLE IF EXISTS #MesRef_07;
CREATE TABLE #MesRef_07 (MesRefVar DATE);
INSERT INTO #MesRef_07
select TO_DATE(ADD_MONTHS(current_date, -7), 'YYYY-MM-01') as MesRefVar
;

DROP TABLE IF EXISTS #baseHabilitada_07;
CREATE TABLE #baseHabilitada_07 (documento VARCHAR, anomes DATE, agregado VARCHAR, anomesCancelamento DATE);
INSERT INTO #baseHabilitada_07
select distinct
d.documento*1 :: float as documento,
TO_DATE(datahabilitada, 'YYYY-MM-01') as anomes,
a.agregado as agregado,
TO_DATE(b.datacancelamento, 'YYYY-MM-01') as anomesCancelamento
from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as a
left join cntcar_work.tb_conectcar_dbo_adesao_stage as b on a.adesaoid = b.adesaoid
left join cntcar_work.tb_conectcar_dbo_veiculo_stage as c on b.veiculoID = c.veiculo_id
left join conectcar.cntcar_work.tb_conectcar_dbo_cliente_stage as d on b.Clienteid = d.Clienteid
where anomes = (SELECT MesRefVar FROM #MesRef_07)
and len(documento) <= 11
;

DROP TABLE IF EXISTS #flagPlacasas_07;
CREATE TABLE #flagPlacasas_07 (documento VARCHAR, flagPlacas FLOAT);
INSERT INTO #flagPlacasas_07
select
documento,
count (documento) as flagPlacas
from #baseHabilitada_07
group by documento
having flagPlacas > 1
order by flagPlacas desc
;

DROP TABLE IF EXISTS #baseHabilitada_07PFCom1PlacaSemPorto;
CREATE TABLE #baseHabilitada_07PFCom1PlacaSemPorto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_07PFCom1PlacaSemPorto
select
a.documento ::float,
a.anomes,
a.agregado,
a.anomescancelamento
from #baseHabilitada_07 a
left join #flagPlacasas_07 b
on a.documento = b.documento
where b.flagPlacas is NULL
;

DROP TABLE IF EXISTS #baseHabilitada_07Porto;
CREATE TABLE #baseHabilitada_07Porto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_07Porto
select
regexp_replace(c.codigointernoparceiro,'[^0-9]+', '') as Documento,
TO_DATE(datahabilitada, 'YYYY-MM-01') as anomes,
p.agregado,
TO_DATE(a.datacancelamento, 'YYYY-MM-01') as anomescancelamento
from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as bh
left outer join cntcar_work.tb_parceiro_dbo_ativacaocontrato_stage ac on ac.adesaoid = bh.adesaoid
left outer join cntcar_work.tb_parceiro_dbo_contrato_stage c on c.contratoid = ac.contratoid
left outer join cntcar_work.vw_redshift_ciclodevida_ativacao a on a.adesaoid = ac.adesaoid
left outer join cntcar_work.vw_ciclodevida_portifolio p on p.adesaoid = bh.adesaoid
where
upper(p.agregado) = 'PORTO SEGURO'
and upper(p.detalhado) not in ('CARROFACIL','CARRO FACIL','CARRO FÁCIL')
and upper(a.nomeplano) <> 'LOCADORA'
and TO_DATE(datahabilitada, 'YYYY-MM-01') = (SELECT MesRefVar FROM #MesRef_07)
;
DROP TABLE IF EXISTS #flagPlacasasPorto_07;
CREATE TABLE #flagPlacasasPorto_07 (documento VARCHAR, flagPlacas FLOAT);
INSERT INTO #flagPlacasasPorto_07
select
documento,
count(documento) as flagPlacas
from #baseHabilitada_07Porto
group by documento
having flagPlacas > 1
;
DROP TABLE IF EXISTS #baseHabilitada_07PFCom1Porto;
CREATE TABLE #baseHabilitada_07PFCom1Porto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_07PFCom1Porto
select
a.documento::float,
a.anomes,
a.agregado,
a.anomescancelamento
from #baseHabilitada_07Porto a
left join #flagPlacasasPorto_07 b on a.documento = b.documento
where b.flagPlacas is null and len(a.documento) <=11
;
DROP TABLE IF EXISTS #basefinal_07;
CREATE TABLE #basefinal_07 (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #basefinal_07
select * from #baseHabilitada_07PFCom1PlacaSemPorto
union
select * from #baseHabilitada_07PFCom1Porto
;
select distinct * from #basefinal_07
"""

In [ ]:
BasePF_Clientes1Placa_07= sqlio.read_sql_query(cte_BasePF_Clientes1Placa_07, conn) 

In [ ]:
print(BasePF_Clientes1Placa_07)

In [ ]:
excluindo_07 = """
DROP TABLE IF EXISTS #MesRef_07;
DROP TABLE IF EXISTS #baseHabilitada_07;
DROP TABLE IF EXISTS #flagPlacasas_07;
DROP TABLE IF EXISTS #baseHabilitada_07PFCom1Porto;
DROP TABLE IF EXISTS #baseHabilitada_07Porto;
DROP TABLE IF EXISTS #baseHabilitada_07PFCom1PlacaSemPorto;
select top 1 * from cntcar_work.tb_redshift_ciclodevida_cliente_stage
"""
sqlio.read_sql_query(excluindo_07, conn) 

#### *06 meses atrás*

In [ ]:
cte_BasePF_Clientes1Placa_06 = """
DROP TABLE IF EXISTS #MesRef_06;
CREATE TABLE #MesRef_06 (MesRefVar DATE);
INSERT INTO #MesRef_06
select TO_DATE(ADD_MONTHS(current_date, -6), 'YYYY-MM-01') as MesRefVar
;

DROP TABLE IF EXISTS #baseHabilitada_06;
CREATE TABLE #baseHabilitada_06 (documento VARCHAR, anomes DATE, agregado VARCHAR, anomesCancelamento DATE);
INSERT INTO #baseHabilitada_06
select distinct
d.documento*1 :: float as documento,
TO_DATE(datahabilitada, 'YYYY-MM-01') as anomes,
a.agregado as agregado,
TO_DATE(b.datacancelamento, 'YYYY-MM-01') as anomesCancelamento
from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as a
left join cntcar_work.tb_conectcar_dbo_adesao_stage as b on a.adesaoid = b.adesaoid
left join cntcar_work.tb_conectcar_dbo_veiculo_stage as c on b.veiculoID = c.veiculo_id
left join conectcar.cntcar_work.tb_conectcar_dbo_cliente_stage as d on b.Clienteid = d.Clienteid
where anomes = (SELECT MesRefVar FROM #MesRef_06)
and len(documento) <= 11
;

DROP TABLE IF EXISTS #flagPlacasas_06;
CREATE TABLE #flagPlacasas_06 (documento VARCHAR, flagPlacas FLOAT);
INSERT INTO #flagPlacasas_06
select
documento,
count (documento) as flagPlacas
from #baseHabilitada_06
group by documento
having flagPlacas > 1
order by flagPlacas desc
;

DROP TABLE IF EXISTS #baseHabilitada_06PFCom1PlacaSemPorto;
CREATE TABLE #baseHabilitada_06PFCom1PlacaSemPorto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_06PFCom1PlacaSemPorto
select
a.documento ::float,
a.anomes,
a.agregado,
a.anomescancelamento
from #baseHabilitada_06 a
left join #flagPlacasas_06 b
on a.documento = b.documento
where b.flagPlacas is NULL
;

DROP TABLE IF EXISTS #baseHabilitada_06Porto;
CREATE TABLE #baseHabilitada_06Porto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_06Porto
select
regexp_replace(c.codigointernoparceiro,'[^0-9]+', '') as Documento,
TO_DATE(datahabilitada, 'YYYY-MM-01') as anomes,
p.agregado,
TO_DATE(a.datacancelamento, 'YYYY-MM-01') as anomescancelamento
from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as bh
left outer join cntcar_work.tb_parceiro_dbo_ativacaocontrato_stage ac on ac.adesaoid = bh.adesaoid
left outer join cntcar_work.tb_parceiro_dbo_contrato_stage c on c.contratoid = ac.contratoid
left outer join cntcar_work.vw_redshift_ciclodevida_ativacao a on a.adesaoid = ac.adesaoid
left outer join cntcar_work.vw_ciclodevida_portifolio p on p.adesaoid = bh.adesaoid
where
upper(p.agregado) = 'PORTO SEGURO'
and upper(p.detalhado) not in ('CARROFACIL','CARRO FACIL','CARRO FÁCIL')
and upper(a.nomeplano) <> 'LOCADORA'
and TO_DATE(datahabilitada, 'YYYY-MM-01') = (SELECT MesRefVar FROM #MesRef_06)
;
DROP TABLE IF EXISTS #flagPlacasasPorto_06;
CREATE TABLE #flagPlacasasPorto_06 (documento VARCHAR, flagPlacas FLOAT);
INSERT INTO #flagPlacasasPorto_06
select
documento,
count(documento) as flagPlacas
from #baseHabilitada_06Porto
group by documento
having flagPlacas > 1
;
DROP TABLE IF EXISTS #baseHabilitada_06PFCom1Porto;
CREATE TABLE #baseHabilitada_06PFCom1Porto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_06PFCom1Porto
select
a.documento::float,
a.anomes,
a.agregado,
a.anomescancelamento
from #baseHabilitada_06Porto a
left join #flagPlacasasPorto_06 b on a.documento = b.documento
where b.flagPlacas is null and len(a.documento) <=11
;
DROP TABLE IF EXISTS #basefinal_06;
CREATE TABLE #basefinal_06 (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #basefinal_06
select * from #baseHabilitada_06PFCom1PlacaSemPorto
union
select * from #baseHabilitada_06PFCom1Porto
;
select distinct * from #basefinal_06
"""

In [ ]:
BasePF_Clientes1Placa_06 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_06, conn) 

In [ ]:
print(BasePF_Clientes1Placa_06)

In [ ]:
excluindo_06 = """
DROP TABLE IF EXISTS #MesRef_06;
DROP TABLE IF EXISTS #baseHabilitada_06;
DROP TABLE IF EXISTS #flagPlacasas_06;
DROP TABLE IF EXISTS #baseHabilitada_06PFCom1Porto;
DROP TABLE IF EXISTS #baseHabilitada_06Porto;
DROP TABLE IF EXISTS #baseHabilitada_06PFCom1PlacaSemPorto;
select top 1 * from cntcar_work.tb_redshift_ciclodevida_cliente_stage
"""
sqlio.read_sql_query(excluindo_06, conn) 

#### *05 meses atrás*

In [ ]:
cte_BasePF_Clientes1Placa_05 = """
DROP TABLE IF EXISTS #MesRef_05;
CREATE TABLE #MesRef_05 (MesRefVar DATE);
INSERT INTO #MesRef_05
select TO_DATE(ADD_MONTHS(current_date, -5), 'YYYY-MM-01') as MesRefVar
;

DROP TABLE IF EXISTS #baseHabilitada_05;
CREATE TABLE #baseHabilitada_05 (documento VARCHAR, anomes DATE, agregado VARCHAR, anomesCancelamento DATE);
INSERT INTO #baseHabilitada_05
select distinct
d.documento*1 :: float as documento,
TO_DATE(datahabilitada, 'YYYY-MM-01') as anomes,
a.agregado as agregado,
TO_DATE(b.datacancelamento, 'YYYY-MM-01') as anomesCancelamento
from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as a
left join cntcar_work.tb_conectcar_dbo_adesao_stage as b on a.adesaoid = b.adesaoid
left join cntcar_work.tb_conectcar_dbo_veiculo_stage as c on b.veiculoID = c.veiculo_id
left join conectcar.cntcar_work.tb_conectcar_dbo_cliente_stage as d on b.Clienteid = d.Clienteid
where anomes = (SELECT MesRefVar FROM #MesRef_05)
and len(documento) <= 11
;

DROP TABLE IF EXISTS #flagPlacasas_05;
CREATE TABLE #flagPlacasas_05 (documento VARCHAR, flagPlacas FLOAT);
INSERT INTO #flagPlacasas_05
select
documento,
count (documento) as flagPlacas
from #baseHabilitada_05
group by documento
having flagPlacas > 1
order by flagPlacas desc
;

DROP TABLE IF EXISTS #baseHabilitada_05PFCom1PlacaSemPorto;
CREATE TABLE #baseHabilitada_05PFCom1PlacaSemPorto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_05PFCom1PlacaSemPorto
select
a.documento ::float,
a.anomes,
a.agregado,
a.anomescancelamento
from #baseHabilitada_05 a
left join #flagPlacasas_05 b
on a.documento = b.documento
where b.flagPlacas is NULL
;

DROP TABLE IF EXISTS #baseHabilitada_05Porto;
CREATE TABLE #baseHabilitada_05Porto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_05Porto
select
regexp_replace(c.codigointernoparceiro,'[^0-9]+', '') as Documento,
TO_DATE(datahabilitada, 'YYYY-MM-01') as anomes,
p.agregado,
TO_DATE(a.datacancelamento, 'YYYY-MM-01') as anomescancelamento
from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as bh
left outer join cntcar_work.tb_parceiro_dbo_ativacaocontrato_stage ac on ac.adesaoid = bh.adesaoid
left outer join cntcar_work.tb_parceiro_dbo_contrato_stage c on c.contratoid = ac.contratoid
left outer join cntcar_work.vw_redshift_ciclodevida_ativacao a on a.adesaoid = ac.adesaoid
left outer join cntcar_work.vw_ciclodevida_portifolio p on p.adesaoid = bh.adesaoid
where
upper(p.agregado) = 'PORTO SEGURO'
and upper(p.detalhado) not in ('CARROFACIL','CARRO FACIL','CARRO FÁCIL')
and upper(a.nomeplano) <> 'LOCADORA'
and TO_DATE(datahabilitada, 'YYYY-MM-01') = (SELECT MesRefVar FROM #MesRef_05)
;
DROP TABLE IF EXISTS #flagPlacasasPorto_05;
CREATE TABLE #flagPlacasasPorto_05 (documento VARCHAR, flagPlacas FLOAT);
INSERT INTO #flagPlacasasPorto_05
select
documento,
count(documento) as flagPlacas
from #baseHabilitada_05Porto
group by documento
having flagPlacas > 1
;
DROP TABLE IF EXISTS #baseHabilitada_05PFCom1Porto;
CREATE TABLE #baseHabilitada_05PFCom1Porto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_05PFCom1Porto
select
a.documento::float,
a.anomes,
a.agregado,
a.anomescancelamento
from #baseHabilitada_05Porto a
left join #flagPlacasasPorto_05 b on a.documento = b.documento
where b.flagPlacas is null and len(a.documento) <=11
;
DROP TABLE IF EXISTS #basefinal_05;
CREATE TABLE #basefinal_05 (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #basefinal_05
select * from #baseHabilitada_05PFCom1PlacaSemPorto
union
select * from #baseHabilitada_05PFCom1Porto
;
select distinct * from #basefinal_05
"""

In [ ]:
BasePF_Clientes1Placa_05= sqlio.read_sql_query(cte_BasePF_Clientes1Placa_05, conn) 

In [ ]:
print(BasePF_Clientes1Placa_05)

In [ ]:
excluindo_05 = """
DROP TABLE IF EXISTS #MesRef_05;
DROP TABLE IF EXISTS #baseHabilitada_05;
DROP TABLE IF EXISTS #flagPlacasas_05;
DROP TABLE IF EXISTS #baseHabilitada_05PFCom1Porto;
DROP TABLE IF EXISTS #baseHabilitada_05Porto;
DROP TABLE IF EXISTS #baseHabilitada_05PFCom1PlacaSemPorto;
select top 1 * from cntcar_work.tb_redshift_ciclodevida_cliente_stage
"""
sqlio.read_sql_query(excluindo_05, conn) 

#### *04 meses atrás*

In [ ]:
cte_BasePF_Clientes1Placa_04 = """
DROP TABLE IF EXISTS #MesRef_04;
CREATE TABLE #MesRef_04 (MesRefVar DATE);
INSERT INTO #MesRef_04
select TO_DATE(ADD_MONTHS(current_date, -4), 'YYYY-MM-01') as MesRefVar
;

DROP TABLE IF EXISTS #baseHabilitada_04;
CREATE TABLE #baseHabilitada_04 (documento VARCHAR, anomes DATE, agregado VARCHAR, anomesCancelamento DATE);
INSERT INTO #baseHabilitada_04
select distinct
d.documento*1 :: float as documento,
TO_DATE(datahabilitada, 'YYYY-MM-01') as anomes,
a.agregado as agregado,
TO_DATE(b.datacancelamento, 'YYYY-MM-01') as anomesCancelamento
from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as a
left join cntcar_work.tb_conectcar_dbo_adesao_stage as b on a.adesaoid = b.adesaoid
left join cntcar_work.tb_conectcar_dbo_veiculo_stage as c on b.veiculoID = c.veiculo_id
left join conectcar.cntcar_work.tb_conectcar_dbo_cliente_stage as d on b.Clienteid = d.Clienteid
where anomes = (SELECT MesRefVar FROM #MesRef_04)
and len(documento) <= 11
;

DROP TABLE IF EXISTS #flagPlacasas_04;
CREATE TABLE #flagPlacasas_04 (documento VARCHAR, flagPlacas FLOAT);
INSERT INTO #flagPlacasas_04
select
documento,
count (documento) as flagPlacas
from #baseHabilitada_04
group by documento
having flagPlacas > 1
order by flagPlacas desc
;

DROP TABLE IF EXISTS #baseHabilitada_04PFCom1PlacaSemPorto;
CREATE TABLE #baseHabilitada_04PFCom1PlacaSemPorto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_04PFCom1PlacaSemPorto
select
a.documento ::float,
a.anomes,
a.agregado,
a.anomescancelamento
from #baseHabilitada_04 a
left join #flagPlacasas_04 b
on a.documento = b.documento
where b.flagPlacas is NULL
;

DROP TABLE IF EXISTS #baseHabilitada_04Porto;
CREATE TABLE #baseHabilitada_04Porto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_04Porto
select
regexp_replace(c.codigointernoparceiro,'[^0-9]+', '') as Documento,
TO_DATE(datahabilitada, 'YYYY-MM-01') as anomes,
p.agregado,
TO_DATE(a.datacancelamento, 'YYYY-MM-01') as anomescancelamento
from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as bh
left outer join cntcar_work.tb_parceiro_dbo_ativacaocontrato_stage ac on ac.adesaoid = bh.adesaoid
left outer join cntcar_work.tb_parceiro_dbo_contrato_stage c on c.contratoid = ac.contratoid
left outer join cntcar_work.vw_redshift_ciclodevida_ativacao a on a.adesaoid = ac.adesaoid
left outer join cntcar_work.vw_ciclodevida_portifolio p on p.adesaoid = bh.adesaoid
where
upper(p.agregado) = 'PORTO SEGURO'
and upper(p.detalhado) not in ('CARROFACIL','CARRO FACIL','CARRO FÁCIL')
and upper(a.nomeplano) <> 'LOCADORA'
and TO_DATE(datahabilitada, 'YYYY-MM-01') = (SELECT MesRefVar FROM #MesRef_04)
;
DROP TABLE IF EXISTS #flagPlacasasPorto_04;
CREATE TABLE #flagPlacasasPorto_04 (documento VARCHAR, flagPlacas FLOAT);
INSERT INTO #flagPlacasasPorto_04
select
documento,
count(documento) as flagPlacas
from #baseHabilitada_04Porto
group by documento
having flagPlacas > 1
;
DROP TABLE IF EXISTS #baseHabilitada_04PFCom1Porto;
CREATE TABLE #baseHabilitada_04PFCom1Porto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_04PFCom1Porto
select
a.documento::float,
a.anomes,
a.agregado,
a.anomescancelamento
from #baseHabilitada_04Porto a
left join #flagPlacasasPorto_04 b on a.documento = b.documento
where b.flagPlacas is null and len(a.documento) <=11
;
DROP TABLE IF EXISTS #basefinal_04;
CREATE TABLE #basefinal_04 (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #basefinal_04
select * from #baseHabilitada_04PFCom1PlacaSemPorto
union
select * from #baseHabilitada_04PFCom1Porto
;
select distinct * from #basefinal_04
"""

In [ ]:
BasePF_Clientes1Placa_04 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_04, conn) 

In [ ]:
print(BasePF_Clientes1Placa_04)

In [ ]:
excluindo_04 = """
DROP TABLE IF EXISTS #MesRef_04;
DROP TABLE IF EXISTS #baseHabilitada_04;
DROP TABLE IF EXISTS #flagPlacasas_04;
DROP TABLE IF EXISTS #baseHabilitada_04PFCom1Porto;
DROP TABLE IF EXISTS #baseHabilitada_04Porto;
DROP TABLE IF EXISTS #baseHabilitada_04PFCom1PlacaSemPorto;
select top 1 * from cntcar_work.tb_redshift_ciclodevida_cliente_stage
"""
sqlio.read_sql_query(excluindo_04, conn) 

#### *03 meses atrás*

In [ ]:
cte_BasePF_Clientes1Placa_03 = """
DROP TABLE IF EXISTS #MesRef_03;
CREATE TABLE #MesRef_03 (MesRefVar DATE);
INSERT INTO #MesRef_03
select TO_DATE(ADD_MONTHS(current_date, -3), 'YYYY-MM-01') as MesRefVar
;

DROP TABLE IF EXISTS #baseHabilitada_03;
CREATE TABLE #baseHabilitada_03 (documento VARCHAR, anomes DATE, agregado VARCHAR, anomesCancelamento DATE);
INSERT INTO #baseHabilitada_03
select distinct
d.documento*1 :: float as documento,
TO_DATE(datahabilitada, 'YYYY-MM-01') as anomes,
a.agregado as agregado,
TO_DATE(b.datacancelamento, 'YYYY-MM-01') as anomesCancelamento
from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as a
left join cntcar_work.tb_conectcar_dbo_adesao_stage as b on a.adesaoid = b.adesaoid
left join cntcar_work.tb_conectcar_dbo_veiculo_stage as c on b.veiculoID = c.veiculo_id
left join conectcar.cntcar_work.tb_conectcar_dbo_cliente_stage as d on b.Clienteid = d.Clienteid
where anomes = (SELECT MesRefVar FROM #MesRef_03)
and len(documento) <= 11
;

DROP TABLE IF EXISTS #flagPlacasas_03;
CREATE TABLE #flagPlacasas_03 (documento VARCHAR, flagPlacas FLOAT);
INSERT INTO #flagPlacasas_03
select
documento,
count (documento) as flagPlacas
from #baseHabilitada_03
group by documento
having flagPlacas > 1
order by flagPlacas desc
;

DROP TABLE IF EXISTS #baseHabilitada_03PFCom1PlacaSemPorto;
CREATE TABLE #baseHabilitada_03PFCom1PlacaSemPorto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_03PFCom1PlacaSemPorto
select
a.documento ::float,
a.anomes,
a.agregado,
a.anomescancelamento
from #baseHabilitada_03 a
left join #flagPlacasas_03 b
on a.documento = b.documento
where b.flagPlacas is NULL
;

DROP TABLE IF EXISTS #baseHabilitada_03Porto;
CREATE TABLE #baseHabilitada_03Porto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_03Porto
select
regexp_replace(c.codigointernoparceiro,'[^0-9]+', '') as Documento,
TO_DATE(datahabilitada, 'YYYY-MM-01') as anomes,
p.agregado,
TO_DATE(a.datacancelamento, 'YYYY-MM-01') as anomescancelamento
from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as bh
left outer join cntcar_work.tb_parceiro_dbo_ativacaocontrato_stage ac on ac.adesaoid = bh.adesaoid
left outer join cntcar_work.tb_parceiro_dbo_contrato_stage c on c.contratoid = ac.contratoid
left outer join cntcar_work.vw_redshift_ciclodevida_ativacao a on a.adesaoid = ac.adesaoid
left outer join cntcar_work.vw_ciclodevida_portifolio p on p.adesaoid = bh.adesaoid
where
upper(p.agregado) = 'PORTO SEGURO'
and upper(p.detalhado) not in ('CARROFACIL','CARRO FACIL','CARRO FÁCIL')
and upper(a.nomeplano) <> 'LOCADORA'
and TO_DATE(datahabilitada, 'YYYY-MM-01') = (SELECT MesRefVar FROM #MesRef_03)
;
DROP TABLE IF EXISTS #flagPlacasasPorto_03;
CREATE TABLE #flagPlacasasPorto_03 (documento VARCHAR, flagPlacas FLOAT);
INSERT INTO #flagPlacasasPorto_03
select
documento,
count(documento) as flagPlacas
from #baseHabilitada_03Porto
group by documento
having flagPlacas > 1
;
DROP TABLE IF EXISTS #baseHabilitada_03PFCom1Porto;
CREATE TABLE #baseHabilitada_03PFCom1Porto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_03PFCom1Porto
select
a.documento::float,
a.anomes,
a.agregado,
a.anomescancelamento
from #baseHabilitada_03Porto a
left join #flagPlacasasPorto_03 b on a.documento = b.documento
where b.flagPlacas is null and len(a.documento) <=11
;
DROP TABLE IF EXISTS #basefinal_03;
CREATE TABLE #basefinal_03 (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #basefinal_03
select * from #baseHabilitada_03PFCom1PlacaSemPorto
union
select * from #baseHabilitada_03PFCom1Porto
;
select distinct * from #basefinal_03
"""

In [ ]:
BasePF_Clientes1Placa_03 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_03, conn) 

In [ ]:
print(BasePF_Clientes1Placa_03)

In [27]:
excluindo_03 = """
DROP TABLE IF EXISTS #MesRef_03;
DROP TABLE IF EXISTS #baseHabilitada_03;
DROP TABLE IF EXISTS #flagPlacasas_03;
DROP TABLE IF EXISTS #baseHabilitada_03PFCom1Porto;
DROP TABLE IF EXISTS #baseHabilitada_03Porto;
DROP TABLE IF EXISTS #baseHabilitada_03PFCom1PlacaSemPorto;
select top 1 * from cntcar_work.tb_redshift_ciclodevida_cliente_stage
"""
sqlio.read_sql_query(excluindo_03, conn) 

c:\Users\raul.correa\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,clienteid,datacadastro,datanascimento,nomecompleto,email,dddcelular,celular,emailspromocionais,permiteenviosms,datatermoconsentimentoitau,...,cep,cidade,estado,canaldevendas,cartaobin,sexoid,estadocivilid,pessoafisica,cartonistaitau,load_date


#### *02 meses atrás*

In [30]:
cte_BasePF_Clientes1Placa_02 = """
DROP TABLE IF EXISTS #MesRef_02;
CREATE TABLE #MesRef_02 (MesRefVar DATE);
INSERT INTO #MesRef_02
select TO_DATE(ADD_MONTHS(current_date, -2), 'YYYY-MM-01') as MesRefVar
;

DROP TABLE IF EXISTS #baseHabilitada_02;
CREATE TABLE #baseHabilitada_02 (documento VARCHAR, anomes DATE, agregado VARCHAR, anomesCancelamento DATE);
INSERT INTO #baseHabilitada_02
select distinct
d.documento*1 :: float as documento,
TO_DATE(datahabilitada, 'YYYY-MM-01') as anomes,
a.agregado as agregado,
TO_DATE(b.datacancelamento, 'YYYY-MM-01') as anomesCancelamento
from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as a
left join cntcar_work.tb_conectcar_dbo_adesao_stage as b on a.adesaoid = b.adesaoid
left join cntcar_work.tb_conectcar_dbo_veiculo_stage as c on b.veiculoID = c.veiculo_id
left join conectcar.cntcar_work.tb_conectcar_dbo_cliente_stage as d on b.Clienteid = d.Clienteid
where anomes = (SELECT MesRefVar FROM #MesRef_02)
and len(documento) <= 11
;

DROP TABLE IF EXISTS #flagPlacasas_02;
CREATE TABLE #flagPlacasas_02 (documento VARCHAR, flagPlacas FLOAT);
INSERT INTO #flagPlacasas_02
select
documento,
count (documento) as flagPlacas
from #baseHabilitada_02
group by documento
having flagPlacas > 1
order by flagPlacas desc
;

DROP TABLE IF EXISTS #baseHabilitada_02PFCom1PlacaSemPorto;
CREATE TABLE #baseHabilitada_02PFCom1PlacaSemPorto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_02PFCom1PlacaSemPorto
select
a.documento ::float,
a.anomes,
a.agregado,
a.anomescancelamento
from #baseHabilitada_02 a
left join #flagPlacasas_02 b
on a.documento = b.documento
where b.flagPlacas is NULL
;

DROP TABLE IF EXISTS #baseHabilitada_02Porto;
CREATE TABLE #baseHabilitada_02Porto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_02Porto
select
regexp_replace(c.codigointernoparceiro,'[^0-9]+', '') as Documento,
TO_DATE(datahabilitada, 'YYYY-MM-01') as anomes,
p.agregado,
TO_DATE(a.datacancelamento, 'YYYY-MM-01') as anomescancelamento
from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as bh
left outer join cntcar_work.tb_parceiro_dbo_ativacaocontrato_stage ac on ac.adesaoid = bh.adesaoid
left outer join cntcar_work.tb_parceiro_dbo_contrato_stage c on c.contratoid = ac.contratoid
left outer join cntcar_work.vw_redshift_ciclodevida_ativacao a on a.adesaoid = ac.adesaoid
left outer join cntcar_work.vw_ciclodevida_portifolio p on p.adesaoid = bh.adesaoid
where
upper(p.agregado) = 'PORTO SEGURO'
and upper(p.detalhado) not in ('CARROFACIL','CARRO FACIL','CARRO FÁCIL')
and upper(a.nomeplano) <> 'LOCADORA'
and TO_DATE(datahabilitada, 'YYYY-MM-01') = (SELECT MesRefVar FROM #MesRef_02)
;
DROP TABLE IF EXISTS #flagPlacasasPorto_02;
CREATE TABLE #flagPlacasasPorto_02 (documento VARCHAR, flagPlacas FLOAT);
INSERT INTO #flagPlacasasPorto_02
select
documento,
count(documento) as flagPlacas
from #baseHabilitada_02Porto
group by documento
having flagPlacas > 1
;
DROP TABLE IF EXISTS #baseHabilitada_02PFCom1Porto;
CREATE TABLE #baseHabilitada_02PFCom1Porto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_02PFCom1Porto
select
a.documento::float,
a.anomes,
a.agregado,
a.anomescancelamento
from #baseHabilitada_02Porto a
left join #flagPlacasasPorto_02 b on a.documento = b.documento
where b.flagPlacas is null and len(a.documento) <=11
;
DROP TABLE IF EXISTS #basefinal_02;
CREATE TABLE #basefinal_02 (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #basefinal_02
select * from #baseHabilitada_02PFCom1PlacaSemPorto
union
select * from #baseHabilitada_02PFCom1Porto
;
select distinct * from #basefinal_02
"""

In [31]:
BasePF_Clientes1Placa_02 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_02, conn) 

c:\Users\raul.correa\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [ ]:
print(BasePF_Clientes1Placa_02)

In [32]:
excluindo_02 = """
DROP TABLE IF EXISTS #MesRef_02;
DROP TABLE IF EXISTS #baseHabilitada_02;
DROP TABLE IF EXISTS #flagPlacasas_02;
DROP TABLE IF EXISTS #baseHabilitada_02PFCom1Porto;
DROP TABLE IF EXISTS #baseHabilitada_02Porto;
DROP TABLE IF EXISTS #baseHabilitada_02PFCom1PlacaSemPorto;
select top 1 * from cntcar_work.tb_redshift_ciclodevida_cliente_stage
"""
sqlio.read_sql_query(excluindo_02, conn) 

c:\Users\raul.correa\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,clienteid,datacadastro,datanascimento,nomecompleto,email,dddcelular,celular,emailspromocionais,permiteenviosms,datatermoconsentimentoitau,...,cep,cidade,estado,canaldevendas,cartaobin,sexoid,estadocivilid,pessoafisica,cartonistaitau,load_date


#### *01 mes atrás*

In [33]:
cte_BasePF_Clientes1Placa_01 = """
DROP TABLE IF EXISTS #MesRef_01;
CREATE TABLE #MesRef_01 (MesRefVar DATE);
INSERT INTO #MesRef_01
select TO_DATE(ADD_MONTHS(current_date, -1), 'YYYY-MM-01') as MesRefVar
;

DROP TABLE IF EXISTS #baseHabilitada_01;
CREATE TABLE #baseHabilitada_01 (documento VARCHAR, anomes DATE, agregado VARCHAR, anomesCancelamento DATE);
INSERT INTO #baseHabilitada_01
select distinct
d.documento*1 :: float as documento,
TO_DATE(datahabilitada, 'YYYY-MM-01') as anomes,
a.agregado as agregado,
TO_DATE(b.datacancelamento, 'YYYY-MM-01') as anomesCancelamento
from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as a
left join cntcar_work.tb_conectcar_dbo_adesao_stage as b on a.adesaoid = b.adesaoid
left join cntcar_work.tb_conectcar_dbo_veiculo_stage as c on b.veiculoID = c.veiculo_id
left join conectcar.cntcar_work.tb_conectcar_dbo_cliente_stage as d on b.Clienteid = d.Clienteid
where anomes = (SELECT MesRefVar FROM #MesRef_01)
and len(documento) <= 11
;

DROP TABLE IF EXISTS #flagPlacasas_01;
CREATE TABLE #flagPlacasas_01 (documento VARCHAR, flagPlacas FLOAT);
INSERT INTO #flagPlacasas_01
select
documento,
count (documento) as flagPlacas
from #baseHabilitada_01
group by documento
having flagPlacas > 1
order by flagPlacas desc
;

DROP TABLE IF EXISTS #baseHabilitada_01PFCom1PlacaSemPorto;
CREATE TABLE #baseHabilitada_01PFCom1PlacaSemPorto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_01PFCom1PlacaSemPorto
select
a.documento ::float,
a.anomes,
a.agregado,
a.anomescancelamento
from #baseHabilitada_01 a
left join #flagPlacasas_01 b
on a.documento = b.documento
where b.flagPlacas is NULL
;

DROP TABLE IF EXISTS #baseHabilitada_01Porto;
CREATE TABLE #baseHabilitada_01Porto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_01Porto
select
regexp_replace(c.codigointernoparceiro,'[^0-9]+', '') as Documento,
TO_DATE(datahabilitada, 'YYYY-MM-01') as anomes,
p.agregado,
TO_DATE(a.datacancelamento, 'YYYY-MM-01') as anomescancelamento
from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as bh
left outer join cntcar_work.tb_parceiro_dbo_ativacaocontrato_stage ac on ac.adesaoid = bh.adesaoid
left outer join cntcar_work.tb_parceiro_dbo_contrato_stage c on c.contratoid = ac.contratoid
left outer join cntcar_work.vw_redshift_ciclodevida_ativacao a on a.adesaoid = ac.adesaoid
left outer join cntcar_work.vw_ciclodevida_portifolio p on p.adesaoid = bh.adesaoid
where
upper(p.agregado) = 'PORTO SEGURO'
and upper(p.detalhado) not in ('CARROFACIL','CARRO FACIL','CARRO FÁCIL')
and upper(a.nomeplano) <> 'LOCADORA'
and TO_DATE(datahabilitada, 'YYYY-MM-01') = (SELECT MesRefVar FROM #MesRef_01)
;
DROP TABLE IF EXISTS #flagPlacasasPorto_01;
CREATE TABLE #flagPlacasasPorto_01 (documento VARCHAR, flagPlacas FLOAT);
INSERT INTO #flagPlacasasPorto_01
select
documento,
count(documento) as flagPlacas
from #baseHabilitada_01Porto
group by documento
having flagPlacas > 1
;
DROP TABLE IF EXISTS #baseHabilitada_01PFCom1Porto;
CREATE TABLE #baseHabilitada_01PFCom1Porto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_01PFCom1Porto
select
a.documento::float,
a.anomes,
a.agregado,
a.anomescancelamento
from #baseHabilitada_01Porto a
left join #flagPlacasasPorto_01 b on a.documento = b.documento
where b.flagPlacas is null and len(a.documento) <=11
;
DROP TABLE IF EXISTS #basefinal_01;
CREATE TABLE #basefinal_01 (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #basefinal_01
select * from #baseHabilitada_01PFCom1PlacaSemPorto
union
select * from #baseHabilitada_01PFCom1Porto
;
select distinct * from #basefinal_01
"""

In [25]:
BasePF_Clientes1Placa_01 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_01, conn) 

c:\Users\raul.correa\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [34]:
print(BasePF_Clientes1Placa_01)

           documento      anomes      agregado anomescancelamento
0        78500966734  2023-02-01          Itaú               None
1        37039910896  2023-02-01      Completo               None
2        13356867709  2023-02-01          Itaú               None
3          461318091  2023-02-01      COMPLETO               None
4        26166297807  2023-02-01        Básico               None
...              ...         ...           ...                ...
1666015  81946155187  2023-02-01  Porto Seguro               None
1666016  17712137870  2023-02-01  Porto Seguro               None
1666017   5179401593  2023-02-01  Porto Seguro               None
1666018   6267805433  2023-02-01  Porto Seguro               None
1666019  49795813072  2023-02-01  Porto Seguro               None

[1666020 rows x 4 columns]


In [35]:
excluindo_01 = """
DROP TABLE IF EXISTS #MesRef_01;
DROP TABLE IF EXISTS #baseHabilitada_01;
DROP TABLE IF EXISTS #flagPlacasas_01;
DROP TABLE IF EXISTS #baseHabilitada_01PFCom1Porto;
DROP TABLE IF EXISTS #baseHabilitada_01Porto;
DROP TABLE IF EXISTS #baseHabilitada_01PFCom1PlacaSemPorto;
select top 1 * from cntcar_work.tb_redshift_ciclodevida_cliente_stage
"""
sqlio.read_sql_query(excluindo_01, conn) 

c:\Users\raul.correa\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,clienteid,datacadastro,datanascimento,nomecompleto,email,dddcelular,celular,emailspromocionais,permiteenviosms,datatermoconsentimentoitau,...,cep,cidade,estado,canaldevendas,cartaobin,sexoid,estadocivilid,pessoafisica,cartonistaitau,load_date


#### *mês atual*

In [37]:
cte_BasePF_Clientes1Placa_00 = """
DROP TABLE IF EXISTS #MesRef_00;
CREATE TABLE #MesRef_00 (MesRefVar DATE);
INSERT INTO #MesRef_00
select TO_DATE(current_date, 'YYYY-MM-01') as MesRefVar
;

DROP TABLE IF EXISTS #baseHabilitada_00;
CREATE TABLE #baseHabilitada_00 (documento VARCHAR, anomes DATE, agregado VARCHAR, anomesCancelamento DATE);
INSERT INTO #baseHabilitada_00
select distinct
d.documento*1 :: float as documento,
TO_DATE(datahabilitada, 'YYYY-MM-01') as anomes,
a.agregado as agregado,
TO_DATE(b.datacancelamento, 'YYYY-MM-01') as anomesCancelamento
from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as a
left join cntcar_work.tb_conectcar_dbo_adesao_stage as b on a.adesaoid = b.adesaoid
left join cntcar_work.tb_conectcar_dbo_veiculo_stage as c on b.veiculoID = c.veiculo_id
left join conectcar.cntcar_work.tb_conectcar_dbo_cliente_stage as d on b.Clienteid = d.Clienteid
where anomes = (SELECT MesRefVar FROM #MesRef_00)
and len(documento) <= 11
;

DROP TABLE IF EXISTS #flagPlacasas_00;
CREATE TABLE #flagPlacasas_00 (documento VARCHAR, flagPlacas FLOAT);
INSERT INTO #flagPlacasas_00
select
documento,
count (documento) as flagPlacas
from #baseHabilitada_00
group by documento
having flagPlacas > 1
order by flagPlacas desc
;

DROP TABLE IF EXISTS #baseHabilitada_00PFCom1PlacaSemPorto;
CREATE TABLE #baseHabilitada_00PFCom1PlacaSemPorto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_00PFCom1PlacaSemPorto
select
a.documento ::float,
a.anomes,
a.agregado,
a.anomescancelamento
from #baseHabilitada_00 a
left join #flagPlacasas_00 b
on a.documento = b.documento
where b.flagPlacas is NULL
;

DROP TABLE IF EXISTS #baseHabilitada_00Porto;
CREATE TABLE #baseHabilitada_00Porto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_00Porto
select
regexp_replace(c.codigointernoparceiro,'[^0-9]+', '') as Documento,
TO_DATE(datahabilitada, 'YYYY-MM-01') as anomes,
p.agregado,
TO_DATE(a.datacancelamento, 'YYYY-MM-01') as anomescancelamento
from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as bh
left outer join cntcar_work.tb_parceiro_dbo_ativacaocontrato_stage ac on ac.adesaoid = bh.adesaoid
left outer join cntcar_work.tb_parceiro_dbo_contrato_stage c on c.contratoid = ac.contratoid
left outer join cntcar_work.vw_redshift_ciclodevida_ativacao a on a.adesaoid = ac.adesaoid
left outer join cntcar_work.vw_ciclodevida_portifolio p on p.adesaoid = bh.adesaoid
where
upper(p.agregado) = 'PORTO SEGURO'
and upper(p.detalhado) not in ('CARROFACIL','CARRO FACIL','CARRO FÁCIL')
and upper(a.nomeplano) <> 'LOCADORA'
and TO_DATE(datahabilitada, 'YYYY-MM-01') = (SELECT MesRefVar FROM #MesRef_00)
;
DROP TABLE IF EXISTS #flagPlacasasPorto_00;
CREATE TABLE #flagPlacasasPorto_00 (documento VARCHAR, flagPlacas FLOAT);
INSERT INTO #flagPlacasasPorto_00
select
documento,
count(documento) as flagPlacas
from #baseHabilitada_00Porto
group by documento
having flagPlacas > 1
;
DROP TABLE IF EXISTS #baseHabilitada_00PFCom1Porto;
CREATE TABLE #baseHabilitada_00PFCom1Porto (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #baseHabilitada_00PFCom1Porto
select
a.documento::float,
a.anomes,
a.agregado,
a.anomescancelamento
from #baseHabilitada_00Porto a
left join #flagPlacasasPorto_00 b on a.documento = b.documento
where b.flagPlacas is null and len(a.documento) <=11
;
DROP TABLE IF EXISTS #basefinal_00;
CREATE TABLE #basefinal_00 (documento VARCHAR, anomes DATE, agregado VARCHAR, anomescancelamento DATE);
INSERT INTO #basefinal_00
select * from #baseHabilitada_00PFCom1PlacaSemPorto
union
select * from #baseHabilitada_00PFCom1Porto
;
select distinct * from #basefinal_00
"""

In [ ]:
BasePF_Clientes1Placa_00 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_00, conn) 

In [ ]:
print(BasePF_Clientes1Placa_00)

In [36]:
excluindo_00 = """
DROP TABLE IF EXISTS #MesRef_00;
DROP TABLE IF EXISTS #baseHabilitada_00;
DROP TABLE IF EXISTS #flagPlacasas_00;
DROP TABLE IF EXISTS #baseHabilitada_00PFCom1Porto;
DROP TABLE IF EXISTS #baseHabilitada_00Porto;
DROP TABLE IF EXISTS #baseHabilitada_00PFCom1PlacaSemPorto;
select top 1 * from cntcar_work.tb_redshift_ciclodevida_cliente_stage
"""
sqlio.read_sql_query(excluindo_00, conn) 

c:\Users\raul.correa\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,clienteid,datacadastro,datanascimento,nomecompleto,email,dddcelular,celular,emailspromocionais,permiteenviosms,datatermoconsentimentoitau,...,cep,cidade,estado,canaldevendas,cartaobin,sexoid,estadocivilid,pessoafisica,cartonistaitau,load_date


### Unindo dataframes

In [49]:
BasePF_Clientes1Placa_09.columns = ['documento', 'anomes', 'agregado', 'anomescancelamento']
BasePF_Clientes1Placa_08.columns = ['documento', 'anomes', 'agregado', 'anomescancelamento']


In [50]:
teste = BasePF_Clientes1Placa_09.join(BasePF_Clientes1Placa_08.set_index('documento'), on='documento', how ='left')

ValueError: columns overlap but no suffix specified: Index(['anomes', 'agregado', 'anomescancelamento'], dtype='object')